In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preparation and Intuition

In [ ]:
#importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import seaborn as sns
sns.set()

In [ ]:
#Loading data of train and test set
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

train

So the image consists of 784 pixels which represents same number of columns and has 1 extra column as label which tells which number it is.

There are 42000 images in train set

Let's split the datas in independent and dependent set

In [ ]:
X_train = train.drop(['label'],axis = 1)
y_train = train['label']

In [ ]:
test

There are 28000 images in test set

In [ ]:
X_test = test

In [ ]:
#Checking for null values in train and test set

print('Number of null values in training set is : ',train.isnull().sum().unique())
print('Number of null values in test set is : ',test.isnull().sum().unique())

So there are no null values in train and test set

## Let's check if there is any imbalanced in data labels

In [ ]:
plt.figure(figsize=(15,8))
plt.hist(y_train,bins=20)
plt.xticks(range(10),fontsize = 15)
plt.yticks(fontsize = 15)
plt.xlabel('Numbers',fontsize = 20)
plt.ylabel('Counts or Frequency',fontsize =20)
plt.show()

Almost each label has enough equal count in training set

## Reshaping and Normalising the digits

Keras takes the digits in tensors form i.e 3d channel of RGB.Since here only grayscale is involved we will reshape the images into (px,px,1) where 1 is number of channel.In case of RGB number of channel is 3.

Values in grayscale range from 0 to 255.Higher values can influence their
weights as compared to lower values weights.Therefore normalising the values between 0 to 1 by dividing every value by 255.

In [ ]:
#Reshaping

X_train_reshaped = X_train.values.reshape(-1,28,28,1)
X_test_reshaped = X_test.values.reshape(-1,28,28,1)

#Normalising

X_train_normalised = X_train_reshaped/255.
X_test_normalised = X_test_reshaped/255.

Let's visualise some digits

In [ ]:
def visualise_digits(cmap = 'gray'):
    fig = plt.figure(figsize=(15,15))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(X_train_normalised[i][:,:,0],cmap = cmap)
        plt.title('Label {}'.format(y_train[i]))
        plt.xticks(ticks = [])
        plt.yticks(ticks = [])
            
            
visualise_digits()

## One hot encoding the target variable i.e. y_train

In [ ]:
#importing library
from tensorflow.keras.utils import to_categorical


#encoding to 10 classes of one hot vectors
y_train_encoded = to_categorical(y_train,num_classes=10)

## Splitting of data into train and validation set

* Let the split be 80-20 train-val split

In [ ]:
#importing library
from sklearn.model_selection import train_test_split

#Splitting the data
X_train_final,X_val,y_train_final,y_val = train_test_split(X_train_normalised,y_train_encoded,test_size = 0.2,random_state = 42)

# Implementation of Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D
from tensorflow.keras.layers import BatchNormalization,Flatten,Dropout
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(input_shape=(28,28,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))

model.add(BatchNormalization())

model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))

model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Dropout(0.3))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())

model.add(BatchNormalization())

model.add(Dense(units=256,activation="relu"))

model.add(Dropout(0.3))

model.add(Dense(units=10, activation="softmax"))

In [ ]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.summary()

In [ ]:
callbacks = [ 
    EarlyStopping(monitor = 'loss', patience = 6), 
    ReduceLROnPlateau(monitor = 'loss', patience = 4)
]

In [ ]:
model.fit(X_train_final,y_train_final,
         batch_size = 64,
         epochs = 100,
         verbose = 1,
         validation_data = (X_val,y_val),
         callbacks = callbacks)

In [ ]:
#Metric on validation set
score = model.evaluate(X_val,y_val,verbose = 0)
print('The loss on validation set is {0} and the accuracy is {1}'.format(round(score[0],3),round(score[1],3)))

# Generating Output

In [ ]:
# predict results
results = model.predict(X_test_normalised)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name='Label')


In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_result.csv",index=False)